In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

def accuracy(model,testloader):
    correct = 0 
    total = 0
    for data in testloader:
            images, labels = data
            if train_on_gpu:
                images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('acc = %.3f' %(correct/total))

class CNN(nn.Module):  
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(128),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(256),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.BatchNorm2d(512),
        nn.ReLU(inplace=True),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
        nn.AvgPool2d(kernel_size=1, stride=1, padding=0),
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(512,10)
        )

    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x


def main():
    # load and transform dataset
    transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                              shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                             shuffle=False, num_workers=2)

    classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    # TODO: Define your optimizer and criterion.
    criterion = nn.CrossEntropyLoss()
    model = CNN()
    if train_on_gpu:
        model.cuda()
    optimizer = optim.SGD(model.parameters(), lr=1e-1,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

    num_epoch = 200
    for epoch in range(num_epoch):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            if train_on_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print('[%d, %5d] loss: %.5f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
        print('Epoch = %d | loss = %.4f'%(epoch+1,running_loss/len(trainloader)))
        scheduler.step()
        accuracy(model,testloader)

    print('Finished Training')

    PATH = './model.pth'
    torch.save(model.state_dict(), PATH)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            if train_on_gpu:
                images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
            100 * correct / total))


if __name__ == "__main__":
    main()


CUDA is available!  Training on GPU ...
Files already downloaded and verified
Files already downloaded and verified
Epoch = 1 | loss = 2.5475
acc = 0.113
Epoch = 2 | loss = 2.2827
acc = 0.115
Epoch = 3 | loss = 2.2759
acc = 0.130
Epoch = 4 | loss = 2.0480
acc = 0.254
Epoch = 5 | loss = 1.7250
acc = 0.376
Epoch = 6 | loss = 1.4172
acc = 0.547
Epoch = 7 | loss = 1.1246
acc = 0.654
Epoch = 8 | loss = 0.9400
acc = 0.693
Epoch = 9 | loss = 0.8232
acc = 0.721
Epoch = 10 | loss = 0.7495
acc = 0.745
Epoch = 11 | loss = 0.6979
acc = 0.772
Epoch = 12 | loss = 0.6562
acc = 0.770
Epoch = 13 | loss = 0.6165
acc = 0.794
Epoch = 14 | loss = 0.5913
acc = 0.780
Epoch = 15 | loss = 0.5747
acc = 0.800
Epoch = 16 | loss = 0.5609
acc = 0.806
Epoch = 17 | loss = 0.5490
acc = 0.798
Epoch = 18 | loss = 0.5303
acc = 0.816
Epoch = 19 | loss = 0.5139
acc = 0.792
Epoch = 20 | loss = 0.5089
acc = 0.812
Epoch = 21 | loss = 0.5027
acc = 0.831
Epoch = 22 | loss = 0.4858
acc = 0.818
Epoch = 23 | loss = 0.4814
acc = 0.